In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import pandas as pd
import pymongo
# import itertools
# from concurrent.futures import ProcessPoolExecutor
# from sklearn import preprocessing
# from sklearn.preprocessing import MinMaxScaler
# from sklearn import neighbors
# from sklearn.externals import joblib
from common.log import *
from common.config import Config
from spider.spider_nasdaq import Spider_nasdaq

CONF = Config().data[0]
MONGODB = CONF['MONGODB']
NASDAQ = CONF['NASDAQ']
CRYPTOCURRENCY = CONF['CRYPTOCURRENCY']
NASDAQ100 = CONF['NASDAQ100']

In [2]:
%%time

spider = Spider_nasdaq()
timeframe = '10y'
ignore = True

# for symbol in NASDAQ100:
for symbol in ['NDX']:
#     if symbol == 'WDAY':
#         ignore = False
#     if ignore:
#         continue
    datafile = spider.get_stock_data(symbol, timeframe)
    print(datafile)

../database/market/NDX.csv
Wall time: 18.6 s


In [3]:
%%time

# symbol = 'TSLA'

stock_df_dict = {}

for symbol in NASDAQ100:
    stock_data_file = '../database/market/%s.csv' % symbol
    stock_df = pd.read_csv(stock_data_file)

    # 筛选字段
#     stock_df = stock_df.loc[:, ['date', 'open', 'close', 'volume']]
    stock_df = stock_df.loc[:, ['date', 'close']]

    # 去掉Nasdaq行情首行的当天行情
    if symbol in NASDAQ:
        stock_df = stock_df.drop([0])

    # 处理Coinmarketcap的早期缺失数据
    if symbol in CRYPTOCURRENCY:
        stock_df = stock_df.replace(to_replace='-', value=pd.np.nan)

    # 抛弃空值异常值
    stock_df.dropna(axis=0, how='any', inplace=True)

    # 格式化日期，和news_df['publish_time']一致
    stock_df['date'] = stock_df['date'].apply(lambda x: pd.Period(x, freq='D'))

    # 转换字段格式
#     stock_df = stock_df.astype(dtype={'volume': 'float64'})

    # 用日期作索引，日期升序排序
    stock_df.set_index(['date'], inplace=True)
    stock_df = stock_df.sort_index(axis=0, ascending=True)

    stock_df['pct_change'] = stock_df.close.pct_change(1)
    
#     stock_df = stock_df['2016-01-01':]
    
    stock_df_dict[symbol] = stock_df

# stock_df
# df = stock_df.copy()

Wall time: 4min 13s


In [17]:
%%time

PROPERTY = 100000
CASH = 100000
hold = []
recode_df = pd.DataFrame(columns=['date', 'ops', 'reason', 'symbol', 'count', 'price', 'buy_price', 'expect', 'profit', 'cash', 'property'])

start_date = '2008-06-02'
start_date = '2017-01-03'
end_date = '2018-05-01'

yesterday = None

show_df = None
show_df = stock_df_dict['NDX'].copy()
show_df['CASH'] = pd.np.nan
show_df['PROPERTY'] = pd.np.nan

for today in pd.period_range(start=start_date, end=end_date, freq='D'):
    if yesterday is None:
        yesterday = today
        continue

    if today not in stock_df_dict['NDX'].index:
        continue

    show_df.loc[today, 'CASH'] = CASH
    PROPERTY = CASH + sum([stock_df_dict[order[1]].loc[yesterday, 'close'] * order[2] for order in hold])
    show_df.loc[today, 'PROPERTY'] = PROPERTY

    for order in hold:
        if stock_df_dict[order[1]].loc[today, 'close'] > order[4]:
            hold.remove(order)
            CASH += stock_df_dict[order[1]].loc[today, 'close'] * order[2]
#             print('%s win SELL %s * %d @ %f' % (today, order[1], order[2], stock_df_dict[order[1]].loc[today, 'close']))
            recode_df = recode_df.append(
                {
                    'date': today,
                    'ops': 'SELL',
                    'reason': 'WIN',
                    'symbol': order[1],
                    'count': order[2],
                    'price': stock_df_dict[order[1]].loc[today, 'close'],
                    'buy_price': order[3],
                    'expect': order[4],
                    'profit': (stock_df_dict[order[1]].loc[today, 'close'] - order[3]) * order[2],
                    'cash': CASH,
                    'property': PROPERTY,
                },
                ignore_index=True
            )

        elif stock_df_dict[order[1]].loc[today, 'close'] / order[4] < 0.75:
            hold.remove(order)
            CASH += stock_df_dict[order[1]].loc[today, 'close'] * order[2]
#             print('%s loss SELL %s * %d @ %f' % (today, order[1], order[2], stock_df_dict[order[1]].loc[today, 'close']))
            recode_df = recode_df.append(
                {
                    'date': today,
                    'ops': 'SELL',
                    'reason': 'LOSS',
                    'symbol': order[1],
                    'count': order[2],
                    'price': stock_df_dict[order[1]].loc[today, 'close'],
                    'buy_price': order[3],
                    'expect': order[4],
                    'profit': (stock_df_dict[order[1]].loc[today, 'close'] - order[3]) * order[2],
                    'cash': CASH,
                    'property': PROPERTY,
                },
                ignore_index=True
            )

        elif today - order[0] > 30:
            hold.remove(order)
            CASH += stock_df_dict[order[1]].loc[today, 'close'] * order[2]
#             print('%s TIMEUP SELL %s * %d @ %f' % (today, order[1], order[2], stock_df_dict[order[1]].loc[today, 'close']))
            recode_df = recode_df.append(
                {
                    'date': today,
                    'ops': 'SELL',
                    'reason': 'TIMEUP',
                    'symbol': order[1],
                    'count': order[2],
                    'price': stock_df_dict[order[1]].loc[today, 'close'],
                    'buy_price': order[3],
                    'expect': order[4],
                    'profit': (stock_df_dict[order[1]].loc[today, 'close'] - order[3]) * order[2],
                    'cash': CASH,
                    'property': PROPERTY,
                },
                ignore_index=True
            )
    
    for symbol in NASDAQ100:
        if today not in stock_df_dict[symbol].index:
            continue
        if stock_df_dict[symbol].loc[today, 'pct_change'] < -0.05:
            if CASH < 10000:
                continue
            buy = int(CASH / 10 / stock_df_dict[symbol].loc[today, 'close'])
#             buy = int(PROPERTY / 2 / stock_df_dict[symbol].loc[today, 'close'])
            CASH -= stock_df_dict[symbol].loc[today, 'close'] * buy
            hold.append([today, symbol, buy, stock_df_dict[symbol].loc[today, 'close'], stock_df_dict[symbol].loc[yesterday, 'close']])
#             print('%s 进场 BUY  %s * %d @ %f EXPECT %f' % (today, symbol, buy, stock_df_dict[symbol].loc[today, 'close'], stock_df_dict[symbol].loc[yesterday, 'close']))
            recode_df = recode_df.append(
                {
                    'date': today,
                    'ops': 'BUY',
                    'reason': 'ENTER',
                    'symbol': symbol,
                    'count': buy,
                    'price': stock_df_dict[symbol].loc[today, 'close'],
                    'buy_price': stock_df_dict[symbol].loc[today, 'close'],
                    'expect': stock_df_dict[symbol].loc[yesterday, 'close'],
                    'profit': 0,
                    'cash': CASH,
                    'property': PROPERTY,
                },
                ignore_index=True
            )
    
    show_df.loc[today, 'CASH'] = CASH
    PROPERTY = CASH + sum([stock_df_dict[order[1]].loc[yesterday, 'close'] * order[2] for order in hold])
    show_df.loc[today, 'PROPERTY'] = PROPERTY

    yesterday = today
            

print(CASH)
print(hold)
print(PROPERTY)

# s = 0
# c = 0
# for symbol in NASDAQ100:
# # for symbol in ['TSLA', 'NVDA', 'BIDU', 'FB']:
#     s_p = stock_df_dict[symbol].loc['2017-01-03', 'close']
#     e_p = stock_df_dict[symbol].loc['2018-01-03', 'close']
#     print(symbol, s_p, e_p, e_p / s_p)
#     s += e_p / s_p
#     c += 1

# print(s / c)
    
s_p = stock_df_dict['NDX'].loc[start_date, 'close']
e_p = stock_df_dict['NDX'].loc[end_date, 'close']
print('NDX', s_p, e_p, e_p / s_p)

# print(recode_df)

21557.1911
[[Period('2018-04-06', 'D'), 'INCY', 45, 64.019999999999996, 83.069999999999993], [Period('2018-04-06', 'D'), 'XLNX', 39, 66.299999999999997, 70.700000000000003], [Period('2018-04-11', 'D'), 'FAST', 81, 51.049999999999997, 54.420000000000002], [Period('2018-04-11', 'D'), 'MELI', 11, 328.69999999999999, 347.82999999999998], [Period('2018-04-18', 'D'), 'AMAT', 100, 54.729999999999997, 57.859999999999999], [Period('2018-04-19', 'D'), 'ALGN', 22, 252.78, 277.05000000000001], [Period('2018-04-19', 'D'), 'AMAT', 99, 51.189999999999998, 54.729999999999997], [Period('2018-04-19', 'D'), 'ASML', 23, 195.09999999999999, 206.31999999999999], [Period('2018-04-19', 'D'), 'KLAC', 40, 102.29000000000001, 108.23999999999999], [Period('2018-04-19', 'D'), 'LRCX', 19, 190.38999999999999, 203.77000000000001], [Period('2018-04-19', 'D'), 'SWKS', 35, 93.879999999999995, 98.959999999999994], [Period('2018-04-24', 'D'), 'INCY', 85, 62.350000000000001, 68.140000000000001], [Period('2018-04-24', 'D'),

In [19]:
recode_df.loc[recode_df['ops']=='SELL']

,date,ops,reason,symbol,count,price,buy_price,expect,profit,cash,property
8,2017-02-06,SELL,TIMEUP,REGN,27,362.00,358.6800,380.9200,89.64,53205.7011,100324
9,2017-02-06,SELL,WIN,HAS,80,94.31,81.5100,86.7600,1024,60750.5011,100324
11,2017-02-09,SELL,WIN,CTXS,111,78.42,72.9099,77.9758,611.621,63440.4211,101402
12,2017-02-15,SELL,WIN,DISH,126,62.89,58.0200,62.2300,613.62,71364.5611,102706
13,2017-02-22,SELL,WIN,QCOM,109,57.10,54.0500,56.9000,332.45,77588.4611,103344
14,2017-02-23,SELL,TIMEUP,QCOM,164,57.14,54.8800,62.8800,370.64,86959.4211,103403
15,2017-02-23,SELL,WIN,ADP,50,101.64,95.2500,100.9900,319.5,92041.4211,103403
19,2017-02-27,SELL,TIMEUP,AAL,113,46.32,46.9500,49.5900,-71.19,72582.1711,103686
22,2017-03-13,SELL,TIMEUP,GILD,90,68.10,66.8300,73.1300,114.3,64992.7111,103124
23,2017-03-16,SELL,WIN,QRTEA,384,20.07,18.8800,19.9500,456.96,72699.5911,103872


In [14]:
%matplotlib inline

import sys
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 10]

df = show_df.dropna(how='any', inplace=False)
df['close'] = df['close'] / df.head(1)['close'].values
df['PROPERTY'] = df['PROPERTY'] / df.head(1)['PROPERTY'].values
# stock_df_dict['NDX']

# ax = df.plot(kind='line', y='close', color='Green')
# ax = df.plot(kind='line', y='PROPERTY', secondary_y=False, color='Red', ax=ax)

# plt.figure(figsize=(10, 6), dpi=80)
# plt.show()

d:\python3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
d:\python3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
